<a href="https://colab.research.google.com/github/JoaoVitorDeOliveira/AWSLocal/blob/master/Glovo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# All dependecies and configurations to create a Spark Environment on Colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# make pyspark importable in the environment
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [25]:
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45255)
Traceback (most recent call last):
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [2]:
# Download and import csv files to Spark Environment on Colab
!wget --quiet --show-progress -O connectivity_status.csv https://raw.githubusercontent.com/JoaoVitorDeOliveira/glovo/main/connectivity_status.csv
df_connectivity_status = sc.read.csv("./connectivity_status.csv", inferSchema=True, header=True)

!wget --quiet --show-progress -O orders.csv https://raw.githubusercontent.com/JoaoVitorDeOliveira/glovo/main/orders.csv
df_orders = sc.read.csv("./orders.csv", inferSchema=True, header=True)

!wget --quiet --show-progress -O polling.csv https://raw.githubusercontent.com/JoaoVitorDeOliveira/glovo/main/polling.csv
df_polling = sc.read.csv("./polling.csv", inferSchema=True, header=True)

connectivity_status 100%[===================>]   2.52M  --.-KB/s    in 0.07s   
orders.csv          100%[===================>] 163.77K  --.-KB/s    in 0.02s   
polling.csv         100%[===================>]  24.30M   153MB/s    in 0.2s    


In [ ]:
df_orders.show()

+---+-------------------+---------+--------------------+
|_c0|order_creation_time| order_id|           device_id|
+---+-------------------+---------+--------------------+
|  0|2020-02-26 13:29:00|102523405|                null|
|  1|2020-02-26 21:13:24|102641477|                null|
|  2|2020-02-26 17:04:33|102563229|                null|
|  3|2020-02-26 20:07:45|102622016|                null|
|  4|2020-02-26 21:19:25|102643455|                null|
|  5|2020-02-26 00:08:19|102452116|d0bc996f-72d2-4ec...|
|  6|2020-02-26 10:34:36|102492697|2bb11f99-ab21-462...|
|  7|2020-02-26 11:32:47|102500373|2aec0e20-e1d8-432...|
|  8|2020-02-26 11:46:17|102503173|37638585-a181-426...|
|  9|2020-02-26 11:38:40|102501909|53d16c33-5980-4ad...|
| 10|2020-02-26 11:57:53|102504038|3c533ccb-8739-49c...|
| 11|2020-02-26 12:26:11|102510286|82fb1d95-c894-449...|
| 12|2020-02-26 12:46:31|102514893|7995c021-8983-4f5...|
| 13|2020-02-26 12:52:04|102515379|1cdc8ef5-3752-420...|
| 14|2020-02-26 13:40:44|102525

In [ ]:
df_connectivity_status.show()

+---+--------------------+-------+--------------------+
|_c0|       creation_time| status|           device_id|
+---+--------------------+-------+--------------------+
|  0|2020-02-26 19:31:...|OFFLINE|00083c70-7f54-432...|
|  1|2020-02-26 19:31:...| ONLINE|00083c70-7f54-432...|
|  2|2020-02-26 20:08:...|OFFLINE|000fec74-a5b4-40f...|
|  3|2020-02-26 20:09:...| ONLINE|000fec74-a5b4-40f...|
|  4|2020-02-26 00:30:...|OFFLINE|001d3c67-99fd-43b...|
|  5|2020-02-26 00:30:...| ONLINE|001d3c67-99fd-43b...|
|  6|2020-02-26 09:02:...|OFFLINE|002315d2-9ec3-468...|
|  7|2020-02-26 09:02:...| ONLINE|002315d2-9ec3-468...|
|  8|2020-02-26 03:20:...|OFFLINE|002ec021-a3e8-473...|
|  9|2020-02-26 03:20:...|OFFLINE|002ec021-a3e8-473...|
| 10|2020-02-26 03:20:...| ONLINE|002ec021-a3e8-473...|
| 11|2020-02-26 03:20:...| ONLINE|002ec021-a3e8-473...|
| 12|2020-02-26 09:41:...|OFFLINE|002ec021-a3e8-473...|
| 13|2020-02-26 09:41:...|OFFLINE|002ec021-a3e8-473...|
| 14|2020-02-26 09:41:...| ONLINE|002ec021-a3e8-

In [ ]:
df_polling.show()

+---+--------------------+--------------------+----------+-----------+
|_c0|       creation_time|           device_id|error_code|status_code|
+---+--------------------+--------------------+----------+-----------+
|  0|2020-02-26 19:16:...|d0460656-95e0-484...|      null|        200|
|  1|2020-02-26 19:16:...|d0460656-95e0-484...|      null|        200|
|  2|2020-02-26 18:31:...|d0460656-95e0-484...|      null|        200|
|  3|2020-02-26 18:31:...|d0460656-95e0-484...|      null|        200|
|  4|2020-02-26 18:31:...|d0460656-95e0-484...|      null|        200|
|  5|2020-02-26 18:30:...|d0460656-95e0-484...|      null|        200|
|  6|2020-02-26 18:30:...|d0460656-95e0-484...|      null|        200|
|  7|2020-02-26 18:30:...|d0460656-95e0-484...|      null|        200|
|  8|2020-02-26 18:30:...|d0460656-95e0-484...|      null|        200|
|  9|2020-02-26 18:30:...|d0460656-95e0-484...|      null|        200|
| 10|2020-02-26 18:30:...|d0460656-95e0-484...|      null|        200|
| 11|2

In [3]:
df_orders_polling_con_status = (df_orders
                                .join(df_polling, 'device_id', 'outer')
                                .withColumnRenamed('creation_time', 'polling_creation_time')
                                .join(df_connectivity_status, 'device_id', 'outer')
                                .withColumnRenamed('creation_time', 'status_creation_time')
                                .drop('_c0'))
df_orders_polling_con_status.show()

+--------------------+-------------------+--------+---------------------+----------+-----------+--------------------+-------+
|           device_id|order_creation_time|order_id|polling_creation_time|error_code|status_code|status_creation_time| status|
+--------------------+-------------------+--------+---------------------+----------+-----------+--------------------+-------+
|004fbfe1-a85a-4ab...|               null|    null|                 null|      null|       null|2020-02-26 07:22:...| ONLINE|
|004fbfe1-a85a-4ab...|               null|    null|                 null|      null|       null|2020-02-26 08:27:...|OFFLINE|
|004fbfe1-a85a-4ab...|               null|    null|                 null|      null|       null|2020-02-26 08:38:...| ONLINE|
|004fbfe1-a85a-4ab...|               null|    null|                 null|      null|       null|2020-02-26 12:44:...|OFFLINE|
|004fbfe1-a85a-4ab...|               null|    null|                 null|      null|       null|2020-02-26 12:49:...| 

In [6]:
df_orders_polling_con_status = df_orders_polling_con_status.where('order_creation_time IS NOT NULL')

In [7]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window

In [ ]:
df_orders_polling_con_status.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- order_creation_time: timestamp (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- creation_time: timestamp (nullable = true)
 |-- error_code: string (nullable = true)
 |-- status_code: integer (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- creation_time: timestamp (nullable = true)
 |-- status: string (nullable = true)



In [ ]:
df_orders_polling_con_status.groupBy(f.window('order_creation_time', '3 minutes')).agg(f.count('order_id').alias('orders')).show(truncate=False)

+------------------------------------------+------+
|window                                    |orders|
+------------------------------------------+------+
|[2020-02-26 12:51:00, 2020-02-26 12:54:00]|26057 |
|[2020-02-26 12:21:00, 2020-02-26 12:24:00]|53595 |
|[2020-02-26 17:51:00, 2020-02-26 17:54:00]|8826  |
|[2020-02-26 22:03:00, 2020-02-26 22:06:00]|35260 |
|[2020-02-26 18:15:00, 2020-02-26 18:18:00]|44331 |
|[2020-02-26 12:18:00, 2020-02-26 12:21:00]|28702 |
|[2020-02-26 17:03:00, 2020-02-26 17:06:00]|16405 |
|[2020-02-26 09:00:00, 2020-02-26 09:03:00]|9362  |
|[2020-02-26 21:33:00, 2020-02-26 21:36:00]|13319 |
|[2020-02-26 23:54:00, 2020-02-26 23:57:00]|478   |
|[2020-02-26 20:12:00, 2020-02-26 20:15:00]|100592|
|[2020-02-26 14:03:00, 2020-02-26 14:06:00]|52663 |
|[2020-02-26 13:51:00, 2020-02-26 13:54:00]|41274 |
|[2020-02-26 21:54:00, 2020-02-26 21:57:00]|277750|
|[2020-02-26 08:03:00, 2020-02-26 08:06:00]|708   |
|[2020-02-26 00:06:00, 2020-02-26 00:09:00]|9396  |
|[2020-02-26

In [8]:
df_enhanced = (df_orders_polling_con_status
               #.withColumn('oct_minus_3_min', f.col("order_creation_time") - f.expr("INTERVAL 3 minutes"))
               #.withColumn('oct_plus_3_min', f.col("order_creation_time") + f.expr("INTERVAL 3 minutes"))
               #.withColumn('oct_minus_1_hour', f.col("order_creation_time") - f.expr("INTERVAL 1 hours"))
               .withColumn("pct_preceding", f.lead("polling_creation_time",1).over(Window.orderBy('polling_creation_time')))
               .withColumn("pct_following", f.lag("polling_creation_time",1).over(Window.orderBy('polling_creation_time'))))

In [8]:
df_enhanced.show()

+--------------------+-------------------+---------+---------------------+----------+-----------+--------------------+------+-------------------+-------------------+-------------------+-------------+-------------+
|           device_id|order_creation_time| order_id|polling_creation_time|error_code|status_code|status_creation_time|status|    oct_minus_3_min|     oct_plus_3_min|   oct_minus_1_hour|pct_preceding|pct_following|
+--------------------+-------------------+---------+---------------------+----------+-----------+--------------------+------+-------------------+-------------------+-------------------+-------------+-------------+
|94fd89c6-6324-465...|2020-02-26 15:14:48|102543610|                 null|      null|       null|                null|  null|2020-02-26 15:11:48|2020-02-26 15:17:48|2020-02-26 14:14:48|         null|         null|
|7fae6206-7778-4d5...|2020-02-26 21:38:32|102647329|                 null|      null|       null|                null|  null|2020-02-26 21:35:32

In [15]:
df_enhanced.printSchema()

root
 |-- device_id: string (nullable = true)
 |-- order_creation_time: timestamp (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- polling_creation_time: timestamp (nullable = true)
 |-- error_code: string (nullable = true)
 |-- status_code: integer (nullable = true)
 |-- status_creation_time: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- oct_minus_3_min: timestamp (nullable = true)
 |-- oct_plus_3_min: timestamp (nullable = true)
 |-- oct_minus_1_hour: timestamp (nullable = true)
 |-- pct_preceding: timestamp (nullable = true)
 |-- pct_following: timestamp (nullable = true)



In [9]:
df_enhanced.createOrReplaceTempView('ORDERS')

## The total count of all polling events Three minutes before the order creation time



In [24]:
query = '''SELECT order_id, count(1) as all_polling_events
           FROM ORDERS
           WHERE polling_creation_time IS NOT NULL
           AND polling_creation_time <= (order_creation_time - INTERVAL 3 minutes) 
           GROUP BY order_id
        '''
sc.sql(query).show()

+---------+------------------+
| order_id|all_polling_events|
+---------+------------------+
|102638473|              1443|
|102513750|               942|
|102609981|              3105|
|102603465|              3036|
|102565868|               465|
|102535727|              3161|
|102651853|               550|
|102602920|               310|
|102594830|               634|
|102551214|              3752|
|102633195|              2669|
|102516534|              2388|
|102650480|              2707|
|102540074|              2036|
|102650281|              3781|
|102648123|              2353|
|102517631|              2804|
|102571599|                41|
|102644931|               730|
|102541072|               413|
+---------+------------------+
only showing top 20 rows



## The total count of all polling events Three minutes after the order creation time

In [25]:
query = '''SELECT order_id, count(1) as all_polling_events
           FROM ORDERS
           WHERE polling_creation_time IS NOT NULL
           AND polling_creation_time <= (order_creation_time + INTERVAL 3 minutes) 
           GROUP BY order_id
        '''
sc.sql(query).show()

+---------+------------------+
| order_id|all_polling_events|
+---------+------------------+
|102638473|              1443|
|102513750|               988|
|102609981|              3128|
|102603465|              3059|
|102565868|               485|
|102535727|              3172|
|102651853|               550|
|102602920|               311|
|102594830|               646|
|102551214|              3798|
|102633195|              2693|
|102516534|              2480|
|102650480|              2730|
|102540074|              2059|
|102650281|              3799|
|102648123|              2375|
|102517631|              2827|
|102571599|                41|
|102644931|               734|
|102541072|               423|
+---------+------------------+
only showing top 20 rows



## The total count of all polling events One hour before the order creation time

In [26]:
query = '''SELECT order_id, count(1) as all_polling_events
           FROM ORDERS
           WHERE polling_creation_time IS NOT NULL
           AND polling_creation_time <= (order_creation_time - INTERVAL 1 hour) 
           GROUP BY order_id
        '''
sc.sql(query).show()

+---------+------------------+
| order_id|all_polling_events|
+---------+------------------+
|102638473|              1443|
|102513750|               500|
|102609981|              2885|
|102603465|              2814|
|102565868|               449|
|102535727|              2941|
|102651853|               439|
|102602920|               270|
|102594830|               621|
|102551214|              3314|
|102633195|              2447|
|102516534|              1516|
|102650480|              2486|
|102540074|              1816|
|102648123|              2155|
|102650281|              3370|
|102517631|              2582|
|102571599|                41|
|102644931|               704|
|102541072|               334|
+---------+------------------+
only showing top 20 rows



## The count of each type of polling status_code Three minutes before the order creation time

In [22]:
query = '''SELECT status_code, count(status_code) as count_status_code
           FROM ORDERS
           WHERE status_code IS NOT NULL
           AND polling_creation_time <= (order_creation_time - INTERVAL 3 minutes) 
           GROUP BY status_code
       '''
sc.sql(query).show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45255)
Traceback (most recent call last):
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45255)
Traceback (most recent call last):
  File "spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above e

Py4JNetworkError: ignored

## The count of each type of polling status_code Three minutes after the order creation time

In [28]:
query = '''SELECT status_code, count(status_code) as count_status_code
           FROM ORDERS
           WHERE status_code IS NOT NULL
           AND polling_creation_time <= (order_creation_time + INTERVAL 3 minutes) 
           GROUP BY status_code
       '''
sc.sql(query).show()

+-----------+-----------------+
|status_code|count_status_code|
+-----------+-----------------+
|        200|          7610333|
|          0|            59718|
|        401|              237|
+-----------+-----------------+



## The count of each type of polling status_code One hour before the order creation time

In [29]:
query = '''SELECT status_code, count(status_code) as count_status_code
           FROM ORDERS
           WHERE status_code IS NOT NULL
           AND polling_creation_time <= (order_creation_time - INTERVAL 1 hour) 
           GROUP BY status_code
       '''
sc.sql(query).show()

+-----------+-----------------+
|status_code|count_status_code|
+-----------+-----------------+
|        200|          6578621|
|          0|            49335|
|        401|              211|
+-----------+-----------------+



## The count of each type of polling error_code Three minutes before the order creation time

In [36]:
query = '''SELECT error_code, count(error_code) as count_error_code
           FROM ORDERS
           WHERE polling_creation_time <= (order_creation_time - INTERVAL 3 minutes)
           AND error_code IS NOT NULL
           GROUP BY error_code
        '''
sc.sql(query).show()

+-------------+----------------+
|   error_code|count_error_code|
+-------------+----------------+
| ECONNABORTED|           40528|
|GENERIC_ERROR|           18137|
+-------------+----------------+



## The count of each type of polling error_code Three minutes after the order creation time

In [35]:
query = '''SELECT error_code, count(error_code) as count_error_code
           FROM ORDERS
           WHERE polling_creation_time <= (order_creation_time + INTERVAL 3 minutes)
           AND error_code IS NOT NULL
           GROUP BY error_code
        '''
sc.sql(query).show()

+-------------+----------------+
|   error_code|count_error_code|
+-------------+----------------+
| ECONNABORTED|           41419|
|GENERIC_ERROR|           18536|
+-------------+----------------+



## The count of each type of polling error_code One hour before the order creation time

In [34]:
query = '''SELECT error_code, count(error_code) as count_error_code
           FROM ORDERS
           WHERE polling_creation_time <= (order_creation_time - INTERVAL 1 hour) 
           AND error_code IS NOT NULL
           GROUP BY error_code
        '''
sc.sql(query).show()

+-------------+----------------+
|   error_code|count_error_code|
+-------------+----------------+
| ECONNABORTED|           33790|
|GENERIC_ERROR|           15756|
+-------------+----------------+



## The count of responses without error codes Three minutes before the order creation time

In [48]:
query = '''SELECT count(1) as responses_without_error
           FROM ORDERS
           WHERE polling_creation_time <= (order_creation_time - INTERVAL 3 minutes)
           AND error_code IS NULL
        '''
sc.sql(query).show()

+-----------------------+
|responses_without_error|
+-----------------------+
|                7509318|
+-----------------------+



## The count of responses without error codes Three minutes after the order creation time

In [49]:
query = '''SELECT count(1) as responses_without_error
           FROM ORDERS
           WHERE polling_creation_time <= (order_creation_time + INTERVAL 3 minutes)
           AND error_code IS NULL
        '''
sc.sql(query).show()

+-----------------------+
|responses_without_error|
+-----------------------+
|                7610333|
+-----------------------+



## The count of responses without error codes One hour before the order creation time

In [52]:
query = '''SELECT count(1) as responses_without_error
           FROM ORDERS
           WHERE polling_creation_time <= (order_creation_time - INTERVAL 1 hour)
           AND error_code IS NULL
        '''
sc.sql(query).show()

+-----------------------+
|responses_without_error|
+-----------------------+
|                6578621|
+-----------------------+



In [58]:
df_enhanced.where('status IS NOT NULL').show()

+--------------------+-------------------+---------+---------------------+----------+-----------+--------------------+-------+--------------------+--------------------+
|           device_id|order_creation_time| order_id|polling_creation_time|error_code|status_code|status_creation_time| status|       pct_preceding|       pct_following|
+--------------------+-------------------+---------+---------------------+----------+-----------+--------------------+-------+--------------------+--------------------+
|4daffa4e-a64d-4a3...|2020-02-26 14:26:46|102534563| 2020-02-26 00:00:...|      null|        200|2020-02-26 22:44:...| ONLINE|2020-02-26 00:00:...|2020-02-26 00:00:...|
|4daffa4e-a64d-4a3...|2020-02-26 14:26:46|102534563| 2020-02-26 00:00:...|      null|        200|2020-02-26 22:44:...|OFFLINE|2020-02-26 00:00:...|2020-02-26 00:00:...|
|4daffa4e-a64d-4a3...|2020-02-26 11:32:58|102500991| 2020-02-26 00:00:...|      null|        200|2020-02-26 22:44:...| ONLINE|2020-02-26 00:00:...|2020-02-